# Thompson Sampling for Gaussian Bandits

## Installation
First, install the required dependencies with pinned versions for reproducibility.

In [ ]:
!pip install numpy==1.24.3 matplotlib==3.7.2 scipy==1.11.1

## Overview

**Thompson Sampling** is a smart way to solve the multi-armed bandit problem. Instead of randomly exploring (like epsilon-greedy) or using confidence bounds (like UCB), it uses a more natural approach: it tries actions based on how likely they are to be the best.

### The Core Idea

The basic idea is simple:
- For each action, keep track of what we know and **how certain** we are
- Actions we're **uncertain** about → might be good, so try them occasionally
- Actions we're **confident** are good → try them often
- Actions we're **confident** are bad → avoid them

Think of it like this: if you're unsure whether a restaurant is good, you might give it a chance. But if you've tried it many times and know it's bad, you'll avoid it.

### How It Works

In Thompson Sampling, we select the action $A_t$ at time $t$ as:

$$A_t = \arg\max_a \tilde Q_t(a),$$

where $\tilde Q_t(a)$ is a **random sample** from what we believe about action $a$.

For Gaussian rewards, we sample from:

$$\tilde Q_t(a) \sim \mathcal N\!\left(q(a)|\mu_{a,t},\,\tau_{a,t}^2\right).$$

**What this means:**
- $\mu_{a,t}$: Our best guess of action $a$'s true reward
- $\tau_{a,t}^2$: How uncertain we are (big number = very uncertain)
- As we try action $a$ more, uncertainty shrinks

### Why This Works

Thompson Sampling naturally balances exploration and exploitation:

1. **Early on (high uncertainty)**:
   - We're unsure about all actions
   - Each action occasionally looks best → gets tried

2. **Over time (learning)**:
   - Good actions: Get higher estimates, become more certain
   - Bad actions: Get lower estimates, become more certain
   - Good actions with low uncertainty → selected most often

3. **Eventually**:
   - We're confident about which action is best
   - Mostly select the optimal action
   - Still occasionally try others (but rarely)

### Advantages

**vs. Epsilon-Greedy:**
- ✅ No need to pick an epsilon value
- ✅ Exploration naturally decreases over time
- ✅ Much better performance

**vs. UCB:**
- ✅ Often works better in practice
- ✅ More flexible for complex problems

### Main Idea

Thompson Sampling achieves great performance with no parameters to tune!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from deepmind_bandits import ThompsonSampling, GaussianBandits, BanditDataAnalyzer

# Set random seed for reproducibility
np.random.seed(42)

## Illustration: Sampling from Posteriors

To build intuition, let's visualize how Thompson Sampling uses uncertainty to drive exploration.

Consider three actions with Gaussian posterior distributions:
- **Action C (Green)**: Highest mean ($\mu=2.0$), low variance ($\sigma=0.3$)
- **Action B (Orange)**: Medium mean ($\mu=1.5$), **high variance** ($\sigma=1.0$)
- **Action A (Blue)**: Lowest mean ($\mu=0.0$), low variance ($\sigma=0.3$)

**Question**: Which action should we choose?

**Greedy answer**: Always choose C (highest mean)

**Thompson Sampling's probabilistic answer**:
- C has the highest mean, so it will be sampled most often
- But B has high uncertainty — occasionally it samples values *higher* than C
- When B samples high, we select it and learn more about it
- As we observe B's rewards, its variance shrinks and we become certain whether it's better or worse than C

This is **optimism under uncertainty** in probabilistic form: we give uncertain actions a chance proportional to the probability they could be best.

In [ ]:
# Three Gaussian distributions
q_ = [0.0, 1.5, 2.0]     # different means
stds = [0.3, 1.0, 0.3]   # different standard deviations
labels = ["A", "B", "C"]
x = np.linspace(-1, 4, 400)

plt.figure(figsize=(10, 5))
for mu, sigma, label in zip(q_, stds, labels):
    y = 1/(sigma*np.sqrt(2*np.pi)) * np.exp(-0.5*((x-mu)/sigma)**2)
    plt.plot(x, y, label=f'{label}: μ={mu}, σ={sigma}', linewidth=2)

plt.title("Three Gaussian Posteriors", fontsize=14, fontweight='bold')
plt.xlabel("Reward Value")
plt.ylabel("Probability Density")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Bayesian Inference for Gaussian Rewards

Thompson Sampling needs to track what we know about each action. For Gaussian rewards, we use **Gaussian distributions** to represent our beliefs.

### The Math (Simplified)

After trying action $a$ multiple times ($n_a$ times) with sample mean $\hat{\mu}_{a,t}$, we update our belief:

**Uncertainty (variance):**
$$\tau_{a,t}^2 = \frac{\sigma^2}{n_a}$$

**Best estimate (mean):**
$$\mu_{a,t}=\hat\mu_{a,t}$$

**What this means:**
- More tries ($n_a$ bigger) → less uncertainty ($\tau_{a,t}^2$ smaller)
- Our best guess is just the average reward we've seen
- Uncertainty shrinks as we collect more data

### Estimating Noise

We don't know the reward noise $\sigma^2$ ahead of time, so we estimate it from all the rewards we've seen:

$$\hat\sigma^2 = \frac{1}{\,N-1\,}\sum_{i=1}^{t-1}(r_i-\bar r)^2$$

This is just the sample variance across all observations.

### Implementation Note

Our `ThompsonSampling` class:
- Uses a flat prior (no initial bias)
- Updates beliefs after each reward
- Estimates noise variance online

## Environment Setup

We create the same 4-action Gaussian bandit environment used in previous notebooks:
- Action 0: Low mean (1.0), low noise
- Action 1: **High mean (2.0), moderate noise** — optimal action
- Action 2: Negative mean (-1.0), low noise — clearly bad
- Action 3: Zero mean (0.0), high noise — unpredictable

This environment lets us compare Thompson Sampling's performance against epsilon-greedy and UCB.

In [ ]:
# Create bandit environment
means = [1.0, 2.0, -1.0, 0.0]
stds = [0.1, 0.2, 0.1, 0.3]
env = GaussianBandits(means, stds)
num_actions = env.num_arms

print(f"Number of actions: {num_actions}")
print(f"True mean rewards: {means}")
print(f"Reward noise (stds): {stds}")
print(f"\nOptimal action: {np.argmax(means)} with mean reward = {max(means):.2f}")
print(f"\nGoal: Learn the optimal action through Bayesian reasoning!")

## Agent Initialization

We create a Thompson Sampling agent with:
- **Flat prior** ($\tau_0^2 = \infty$): No initial bias, let data drive learning
- **Initial Q-values**: 0.0 (neutral starting point)
- **Prior mean** $\mu_0 = 0.0$: Center of flat prior (doesn't matter with $\tau_0^2=\infty$)

The agent will:
1. Maintain Gaussian posterior $\mathcal{N}(\mu_{a,t}, \tau_{a,t}^2)$ for each action
2. At each step, sample from all posteriors and select action with highest sample
3. Update posteriors based on observed rewards
4. Naturally explore uncertain actions and exploit confident good actions

In [ ]:
# Create Thompson Sampling agent with flat prior
agent = ThompsonSampling(num_actions=num_actions, initial_q=0.0, tau0_var=np.inf, mu0=0.0)

# Create analyzer for tracking performance
analyzer = BanditDataAnalyzer(means, num_actions)

print(f"Agent Configuration:")
print(f"  Prior: Flat (tau0_var = inf) — no initial bias")
print(f"  Initial posterior means: {agent.means}")
print(f"  Initial posterior variances: {agent.tau}")
print(f"\nThe agent starts with maximum uncertainty about all actions.")

## Training Loop

Now we run the Thompson Sampling algorithm for 1000 time steps. At each step:

1. **Sample from posteriors**: Draw $\tilde{Q}_t(a) \sim \mathcal{N}(\mu_{a,t}, \tau_{a,t}^2)$ for each action $a$
2. **Select action**: Choose $A_t = \arg\max_a \tilde{Q}_t(a)$ (action with highest sampled value)
3. **Observe reward**: Execute action in environment, receive reward $R_t \sim \mathcal{N}(q(A_t), \sigma^2)$
4. **Update posteriors**: 
   - Update pooled variance estimate $\hat{\sigma}^2$ with new reward
   - Update sample mean $\hat{\mu}_{A_t,t}$ for selected action
   - Recompute posterior parameters $\mu_{A_t,t+1}$ and $\tau_{A_t,t+1}^2$
5. **Track metrics**: Record for analysis

Over time, we expect:
- Posterior means $\mu_{a,t}$ to converge to true means $q(a)$
- Posterior variances $\tau_{a,t}^2$ to shrink toward zero (for frequently selected actions)
- Optimal action to be selected with increasing frequency
- Suboptimal regret growth to be logarithmic $O(\log T)$

In [ ]:
T = 1000  # Number of time steps

for t in range(T):
    # Agent samples from posteriors and selects action
    action = agent.select_action()
    
    # Execute action in environment and observe reward
    reward = env.pull_arm(action)
    
    # Update posterior distributions (Bayesian inference)
    agent.update_values(action, reward)
    
    # Track performance
    analyzer.update_and_analyze(action, reward)

print(f"Training completed: {T} time steps\n")
print(f"Final Results:")
print(f"  Posterior means: {agent.means}")
print(f"  Posterior variances: {agent.tau}")
print(f"  Action selection counts: {agent.N}")
print(f"  Pooled variance estimate: {agent.pooled_var:.4f}")
print(f"\nCompare posterior means to true means: {means}")

## Results Analysis

Let's analyze Thompson Sampling's performance and compare it to epsilon-greedy and UCB.

In [ ]:
# Finalize analysis
analyzer.finalize_analysis()

### Q-Value Progression Over Time

This plot shows how posterior means (Q-values) evolved during learning:
- **Solid colored lines**: Posterior mean $\mu_{a,t}$ for each action
- **Dashed lines**: True mean rewards (ground truth)
- **Red arrows**: Action switches

**What to look for:**
- Posterior means should converge to true means (solid → dashed)
- Frequently selected actions converge faster (more data)
- Bad actions (action 2) may not converge if rarely selected (efficient!)
- Early exploration → many switches → rapid learning
- Late exploitation → few switches → stable optimal behavior

In [ ]:
analyzer.plot_Qvalue()

### Cumulative Regret

**Regret** measures the total reward lost by not always selecting the optimal action:

$$\text{Regret}_T = \sum_{t=1}^{T} \left(q^* - q(A_t)\right)$$

where $q^* = \max_a q(a)$ is the optimal mean reward.

**What to look for:**
- **Slope**: Should flatten over time (sublinear growth)
- **Shape**: Logarithmic $O(\log T)$ → curve flattens significantly
- **Comparison**: 
  - Epsilon-greedy: Linear growth $O(T)$ (never stops exploring)
  - UCB: Logarithmic $O(\log T)$ (similar theoretical bound)
  - Thompson Sampling: Logarithmic $O(\log T)$ with better constants in practice

Thompson Sampling typically achieves lower regret than UCB in finite horizons due to better constant factors.

In [ ]:
analyzer.plot_regret()

### Cumulative Reward

This shows total accumulated reward over time:
- **Black line**: Total cumulative reward across all actions
- **Dashed lines**: Cumulative reward per action

**What to look for:**
- Steeper slope = higher reward rate (better performance)
- Action 1's line (optimal) should dominate and have steepest slope
- Total reward should grow steadily, accelerating as we converge to optimal action
- Compare slope to theoretical maximum: $q^* \times T = 2.0 \times 1000 = 2000$

In [ ]:
analyzer.plot_cumulative_reward()

## Performance Summary

In [ ]:
print("=" * 60)
print("THOMPSON SAMPLING PERFORMANCE SUMMARY")
print("=" * 60)

print(f"\nExperiment Parameters:")
print(f"  Total time steps: {T}")
print(f"  Prior: Flat (uninformative)")
print(f"  Pooled variance estimate: {agent.pooled_var:.4f}")

print(f"\nLearned Posterior Distributions:")
for a in range(num_actions):
    mu = agent.means[a]
    tau_sq = agent.tau[a]
    true_mean = means[a]
    error = abs(mu - true_mean)
    count = agent.N[a]
    pct = 100 * count / T
    marker = " ← OPTIMAL" if a == np.argmax(means) else ""
    print(f"  Action {a}: μ={mu:6.3f}, τ²={tau_sq:.4f} (true={true_mean:5.2f}, error={error:.3f}) | "
          f"Selected {count:4d} times ({pct:5.1f}%){marker}")

optimal_action = np.argmax(means)
optimal_selections = agent.N[optimal_action]
optimal_pct = 100 * optimal_selections / T

print(f"\nOptimal Action Performance:")
print(f"  Optimal action: {optimal_action} (true mean = {means[optimal_action]:.2f})")
print(f"  Times selected: {optimal_selections}/{T} ({optimal_pct:.1f}%)")

if optimal_pct >= 85:  # Thompson Sampling should achieve >85% for optimal action
    print(f"  ✓ Excellent! Converged to optimal action with high probability.")
elif optimal_pct >= 70:
    print(f"  ✓ Good convergence to optimal action.")
else:
    print(f"  ⚠ Could improve - may need more time steps or different initialization.")

print(f"\nRegret Analysis:")
print(f"  Final cumulative regret: {analyzer.regret[-1]:.2f}")
print(f"  Average regret per step: {analyzer.regret[-1]/T:.3f}")
print(f"  Expected behavior: Logarithmic growth O(log T)")

print(f"\nPosterior Uncertainty:")
for a in range(num_actions):
    sigma = np.sqrt(agent.tau[a])
    print(f"  Action {a}: σ = {sigma:.4f} (std dev of posterior)")
print(f"\nLower variance = higher confidence. Rarely-selected actions retain high variance.")

print("\n" + "=" * 60)

## Visualize Posterior Distributions

This plot shows the **final posterior belief** about each action's reward:
- **Solid curves**: Posterior distributions $\mathcal{N}(\mu_{a,T}, \tau_{a,T}^2)$
- **Dashed vertical lines**: True mean rewards

**What to look for:**
- **Width** (variance): Narrow = high confidence, Wide = high uncertainty
- **Center** (mean): Should align with dashed line (true mean)
- **Optimal action** (action 1): Should have narrow distribution centered at 2.0
- **Rarely-selected actions**: May have wider distributions (less data)
- **Bad actions**: Should have narrow distributions centered at low values (confidently bad)

This visualization shows how Thompson Sampling's belief state drives its behavior: 
- When distributions overlap significantly → exploration still needed
- When optimal action's distribution is clearly highest and narrow → confident exploitation

In [ ]:
# Plot final posterior distributions
plt.figure(figsize=(12, 5))

# Determine plot range
all_means = means + list(agent.means)
x_min = min(all_means) - 2
x_max = max(all_means) + 2
x = np.linspace(x_min, x_max, 500)

for a in range(num_actions):
    mu = agent.means[a]
    sigma = np.sqrt(agent.tau[a])
    
    # Gaussian PDF
    y = 1/(sigma*np.sqrt(2*np.pi)) * np.exp(-0.5*((x-mu)/sigma)**2)
    label_suffix = " (OPTIMAL)" if a == np.argmax(means) else ""
    plt.plot(x, y, label=f'Action {a}: μ={mu:.2f}, σ={sigma:.2f}{label_suffix}', linewidth=2)
    
    # Mark true mean
    plt.axvline(x=means[a], color=f'C{a}', linestyle='--', alpha=0.3, linewidth=1.5)

plt.title("Final Posterior Distributions (Thompson Sampling)", fontsize=14, fontweight='bold')
plt.xlabel("Reward Value")
plt.ylabel("Probability Density")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Dashed vertical lines show true means for comparison.")
print("Narrow distributions = high confidence, Wide distributions = high uncertainty.")

## Sampling Behavior Demonstration

To understand how Thompson Sampling selects actions, let's simulate 10 decision steps using the final learned posteriors:

At each trial:
1. Sample once from each action's posterior: $\tilde{Q}(a) \sim \mathcal{N}(\mu_a, \tau_a^2)$
2. Select action with highest sample: $A = \arg\max_a \tilde{Q}(a)$

**What to expect:**
- Optimal action (action 1) should be selected most often (high mean, low variance)
- Occasionally other actions may be selected due to random sampling
- High-variance actions have more variable samples → occasional exploration
- This probabilistic behavior is the essence of Thompson Sampling

**Note**: This uses the *final* learned posteriors. Early in training, all actions would be selected more evenly due to high uncertainty.

In [ ]:
# Demonstrate how sampling works
print("=" * 70)
print("SAMPLING FROM FINAL POSTERIORS (10 TRIALS)")
print("=" * 70)
print("\nThis demonstrates Thompson Sampling's probabilistic action selection:\n")

selection_counts = np.zeros(num_actions, dtype=int)

for trial in range(10):
    samples = []
    for a in range(num_actions):
        sample = np.random.normal(agent.means[a], np.sqrt(agent.tau[a]))
        samples.append(sample)
    
    selected = np.argmax(samples)
    selection_counts[selected] += 1
    
    samples_str = [f'{s:5.2f}' for s in samples]
    marker = " ← OPTIMAL" if selected == np.argmax(means) else ""
    print(f"Trial {trial+1:2d}: Samples = [{', '.join(samples_str)}], Selected = {selected}{marker}")

print(f"\nSelection frequency: {selection_counts}")
print(f"True optimal action: {np.argmax(means)}")
print(f"\nOptimal action selected {selection_counts[np.argmax(means)]}/10 times in this demo.")
print("=" * 70)

## Key Takeaways

### What We Learned

1. ✅ **Bayesian Exploration Works**: Thompson Sampling naturally balances exploration and exploitation through probabilistic reasoning
2. ✅ **Efficient Learning**: Achieves logarithmic regret $O(\log T)$ like UCB, often with better constants
3. ✅ **Adaptive Exploration**: Exploration decreases automatically as uncertainty decreases (no need to tune $\epsilon$)
4. ✅ **Optimal Convergence**: Posterior distributions converge to true reward distributions
5. ✅ **Probability Matching**: Selects each action with probability proportional to it being optimal

### Advantages of Thompson Sampling

**vs. Epsilon-Greedy:**
- ✅ No parameter tuning ($\epsilon$ is hard to set correctly)
- ✅ Logarithmic vs linear regret ($O(\log T)$ vs $O(T)$)
- ✅ Exploration driven by uncertainty, not randomness
- ✅ Naturally decreasing exploration over time

**vs. UCB:**
- ✅ Better empirical performance (lower constant factors in regret bound)
- ✅ More natural extension to complex problems (contextual bandits, RL)
- ✅ Handles non-stationary environments better (can use dynamic priors)
- ✅ Principled Bayesian framework

### When to Use Thompson Sampling

**Ideal scenarios:**
- ✅ You can model reward distributions (Gaussian, Bernoulli, etc.)
- ✅ You want state-of-the-art empirical performance
- ✅ Problem may extend to contextual bandits or RL (Thompson Sampling generalizes well)
- ✅ You prefer a principled Bayesian approach
- ✅ You want automatic exploration-exploitation balance

**Avoid when:**
- ⚠️ Reward distribution is unknown and can't be approximated
- ⚠️ You need provable worst-case guarantees (UCB has tighter theory)
- ⚠️ Extreme computational constraints (sampling can be slower than UCB's deterministic selection)

### Practical Recommendations

1. **Start with flat priors** ($\tau_0^2 = \infty$) unless you have genuine prior knowledge
2. **For Bernoulli rewards** (e.g., click-through rates), use Beta-Bernoulli Thompson Sampling
3. **For unknown noise variance**, use pooled variance estimation as shown here
4. **For non-stationary problems**, use discounted posteriors or sliding windows
5. **For production systems**, Thompson Sampling is industry standard (widely used in A/B testing, recommendation systems, ad placement)

### Comparison Summary

| Algorithm | Regret | Exploration | Tuning | Empirical Performance |
|-----------|--------|-------------|--------|----------------------|
| Epsilon-Greedy | $O(T)$ | Random | Hard ($\epsilon$) | Poor |
| UCB | $O(\log T)$ | Deterministic | Easy ($c$) | Good |
| **Thompson Sampling** | $O(\log T)$ | Probabilistic | **None** | **Excellent** |

**Bottom line**: Thompson Sampling is the gold standard for bandit problems where you can model the reward distribution. It combines strong theoretical guarantees with excellent empirical performance and no parameter tuning.